In [49]:
import requests
import re
import wget
import os

from bs4 import BeautifulSoup


In [50]:
def parse(url, parser="html.parser"):
    dump = requests.get(url)
    return BeautifulSoup(dump.content, parser)

def parse_href(tag):
    return tag.attrs["href"]

def sde_match(s):
    return (s[:4] == "sde-" and "TRANQUILITY/" in s) or ()
    
def valid_file_url(url):
    return len(url.split("/")[-1]) > 0

def extract_file(url):
    return url.split("/")[-1]

def download_if_not_exists(url, dest):
    dest = dest if dest[-1] == "/" else f"{dest}/"
    try:
        if not os.path.exists(dest):
            print(f"Creating directory: {dest}")
            os.mkdir(dest)

        file = extract_file(url)
        loc = dest + file
        if not os.path.exists(loc):
            print(f"Downloading\n\turl: {url} \n\tto:{loc}")
            wget.download(url, loc)
        return True
    except KeyboardInterrupt:
        raise()
    except:
        return False


In [53]:
base_url = "https://www.fuzzwork.co.uk/dump/"
store_folder = "/Users/toby/Projects/eve/data/"

dump = parse(base_url)
folders = sorted(filter(sde_match, map(parse_href, dump.find_all("a"))), reverse = True)
failed = []
for folder in folders:
    folder_content = parse(base_url + folder)
    files = map(parse_href, folder_content.find_all("a"))
    files = filter(lambda x: "." in x, files)
    for f in files:
        url = base_url + folder + f
        dest = store_folder + folder
        downloaded =  download_if_not_exists(url, dest)
        if not downloaded:
            failed.append(url)


Downloading
	url: https://www.fuzzwork.co.uk/dump/sde-20200511-TRANQUILITY/postgres-20200511-TRANQUILITY.dmp.bz2 
	to:/Users/toby/Projects/eve/data/sde-20200511-TRANQUILITY/postgres-20200511-TRANQUILITY.dmp.bz2
Downloading
	url: https://www.fuzzwork.co.uk/dump/sde-20190516-TRANQUILITY/evesde.bacpac 
	to:/Users/toby/Projects/eve/data/sde-20190516-TRANQUILITY/evesde.bacpac
Downloading
	url: https://www.fuzzwork.co.uk/dump/sde-20190516-TRANQUILITY/eve.db 
	to:/Users/toby/Projects/eve/data/sde-20190516-TRANQUILITY/eve.db


In [48]:
failed


['https://www.fuzzwork.co.uk/dump/sde-20190516-TRANQUILITY/evesde.bacpac',
 'https://www.fuzzwork.co.uk/dump/sde-20190516-TRANQUILITY/eve.db']